### Upscaling Alamy Images

In [8]:
import cv2
height = width = 640
dim = (width, height)

img = cv2.imread("13000.jpg")
resized_img = cv2.resize(img, dim, cv2.INTER_CUBIC)
cv2.imshow("Resized Image", resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
import os
import shutil
from tqdm import tqdm
import xml.etree.ElementTree as ET

In [16]:
ANNO_ORI_DIR = r"F:\Minor Data Collection\Final Image Data\Monument Original\Original Image Dataset\Annotations"
CURR_IMG_DIR = r"F:\Minor Data Collection\Resized Original Image\Alamy\JPEGImages"
OUTPUT_ANNO_DIR = r"F:\Minor Data Collection\Resized Original Image\Alamy\Annotations"

In [17]:
for file_name in tqdm(os.listdir(CURR_IMG_DIR)):
    img_file_name = file_name.split('.')[0]
    anno_file_name = img_file_name + '.xml'
    src_path = os.path.join(ANNO_ORI_DIR, anno_file_name)
    dst_path = os.path.join(OUTPUT_ANNO_DIR, anno_file_name)
    shutil.copy(src_path, dst_path)

100%|██████████████████████████████████████████████████████████████████████████████| 1984/1984 [00:24<00:00, 81.25it/s]


### Take an image from original Alamy data and Upscale to 512 and relocate it to new directory

In [19]:
BASE_DIR = r"F:\Minor Data Collection\Resized Original Image\Alamy"
OUT_DIR = r"F:\Minor Data Collection\Resized Original Image\Alamy Upscaled"
ANNO_PATH = "Annotations"
IMG_PATH = "JPEGImages"

In [28]:
height = width = 512
dim = (width, height)
for img_file in os.listdir(os.path.join(BASE_DIR, IMG_PATH)):
    img = cv2.imread(os.path.join(BASE_DIR, IMG_PATH, img_file))
    resized_img = cv2.resize(img, dim, cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(OUT_DIR, IMG_PATH, img_file), resized_img)
    
    xml_input_file = os.path.join(BASE_DIR, ANNO_PATH, img_file.split('.')[0] + '.xml')
    xml_file = ET.parse(xml_input_file)
    root = xml_file.getroot()
    
    root.find('size').find('width').text = str(width)
    root.find('size').find('height').text = str(height)
    all_objects = root.findall('object')
    
    for member in all_objects:
        xmin= int(member[4][0].text)
        ymin= int(member[4][1].text)
        xmax= int(member[4][2].text)
        ymax= int(member[4][3].text)
        
        member[4][0].text = str(int((float(xmin) * (512./300.))))
        member[4][1].text = str(int((float(ymin) * (512./300.))))
        member[4][2].text = str(int((float(xmax) * (512./300.))))
        member[4][3].text = str(int((float(ymax) * (512./300.))))
        
    anno_output_path = os.path.join(OUT_DIR, ANNO_PATH, img_file.split('.')[0] + '.xml')
    xml_file.write(anno_output_path)
    
    

### Taking above approach to resize and maintain the original images and its annotations

In [29]:
BASE_DIR = r"F:\Minor Data Collection\All Images\Original Images and Annotations"
OUT_DIR = r"F:\Minor Data Collection\All Images\Original Images and Annotations Resized"
ANNO_PATH = "Annotations"
IMG_PATH = "JPEGImages"

In [35]:
height = width = 512
dim = (width, height)
for img_file in tqdm(os.listdir(os.path.join(BASE_DIR, IMG_PATH))):
    img = cv2.imread(os.path.join(BASE_DIR, IMG_PATH, img_file))
    input_height, input_width, _ = img.shape
    
    resized_img = cv2.resize(img, dim, cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(OUT_DIR, IMG_PATH, img_file), resized_img)
    
    xml_input_file = os.path.join(BASE_DIR, ANNO_PATH, img_file.split('.')[0] + '.xml')
    xml_file = ET.parse(xml_input_file)
    root = xml_file.getroot()
    
    root.find('size').find('width').text = str(width)
    root.find('size').find('height').text = str(height)
    all_objects = root.findall('object')
    
    for member in all_objects:
        xmin= int(member[4][0].text)
        ymin= int(member[4][1].text)
        xmax= int(member[4][2].text)
        ymax= int(member[4][3].text)
        
        member[4][0].text = str(int((float(xmin) / float(input_width)) * width))
        member[4][1].text = str(int((float(ymin) / float(input_height)) * height))
        member[4][2].text = str(int((float(xmax) / float(input_width)) * width))
        member[4][3].text = str(int((float(ymax) / float(input_height)) * height))
        
    anno_output_path = os.path.join(OUT_DIR, ANNO_PATH, img_file.split('.')[0] + '.xml')
    xml_file.write(anno_output_path)

100%|██████████████████████████████████████████████████████████████████████████████| 6185/6185 [20:00<00:00,  5.15it/s]
